**Set environment**

In [6]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [7]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL45
ls ${FD_RES}/results/${ASSAY}/coverage/track_normalized

OL13_20220512.hg38.raw.Input.mean.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.mean.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.raw.pLog2FC.mean.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.mean.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep1.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep2.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep3.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep4.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep5.stranded_pos.bed.gz
OL43_20221003.hg38.raw.Input.rep6.stranded_pos.bed.gz
OL43_20221003.hg38.ra

In [8]:
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FP_INP1=$(ls ${FD_INP}/${PREFIX}*hg38*raw*Input*mean*bed.gz)
FP_INP2=$(ls ${FD_INP}/${PREFIX}*hg38*raw*Output*mean*bed.gz)
FN_INP1=$(basename ${FP_INP1})
FN_INP2=$(basename ${FP_INP2})
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FN_OUT=${FN_INP1/Input/pLog2FC}
FP_OUT=${FD_OUT}/${FN_OUT}

echo ${FN_INP1}
echo 
echo ${FN_INP2}
echo
echo ${FN_OUT}

OL45_20220927.hg38.raw.Input.mean.stranded_pos.bed.gz

OL45_20220927.hg38.raw.Output.mean.stranded_pos.bed.gz

OL45_20220927.hg38.raw.pLog2FC.mean.stranded_pos.bed.gz


## RUN

In [9]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_perbase_pLog2FC_TMPRA_OL45.%a.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL45
SAMPLE_GROUPS=("Input" "Output")

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FP_INP1=$(ls ${FD_INP}/${PREFIX}*hg38*raw*Input*mean*bed.gz)
FP_INP2=$(ls ${FD_INP}/${PREFIX}*hg38*raw*Output*mean*bed.gz)
FN_INP1=$(basename ${FP_INP1})
FN_INP2=$(basename ${FP_INP2})

FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized
FN_OUT=${FN_INP1/Input/pLog2FC}
FP_OUT=${FD_OUT}/${FN_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input file: " ${FP_INP1}
echo
echo "show first few lines of file"
zcat ${FP_INP1} | head -5
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input file: " ${FP_INP2}
echo
echo "show first few lines of file"
zcat ${FP_INP2} | head -5
echo

### execute
bedtools unionbedg -i ${FP_INP1} ${FP_INP2} \
| awk '{
    plog2fc = log($5 + 1)/log(2) - log($4 + 1)/log(2);
    print $1 "\t" $2 "\t" $3 "\t" $4 "\t" $5 "\t" plog2fc;
  }' \
| gzip -c \
> ${FP_OUT}
  
### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output file: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574732


## Check

In [11]:
cat ${FD_LOG}/coverage_perbase_pLog2FC_TMPRA_OL45.0.txt

Hostname:           x3-01-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+15:03:25

++++++++++++++++++++++++++++++++++++++
Input file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage/track_normalized/OL45_20220927.hg38.raw.Input.mean.stranded_pos.bed.gz

show first few lines of file
chr11	4505501	4505502	15.5307
chr11	4505502	4505503	15.5307
chr11	4505503	4505504	15.5307
chr11	4505504	4505505	15.5307
chr11	4505505	4505506	15.5307

++++++++++++++++++++++++++++++++++++++
Input file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage/track_normalized/OL45_20220927.hg38.raw.Output.mean.stranded_pos.bed.gz

show first few lines of file
chr11	4505501	4505502	9.36921
chr11	4505502	4505503	9.36921
chr11	4505503	4505504	9.36921
chr11	4505504	4505505	9.36921
chr11	4505505	4505506	9.36921

++++++++++++++++++++++++++++++++++++++
Output file:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/resu